# 1.3. Основы синтаксиса и структур в Python  
*Выполнения заданий представлены Кузиной Екатериной*

## Задание 1
Необходимо спарсить данные о вакансиях python разработчиков с сайта hh.ru, введя в поиск “python разработчик” и указав, что мы рассматриваем все регионы. Необходимо спарсить:
 - Название вакансии
 - Требуемый опыт работы
 - Заработную плату
 - Регион  

И сохранить эти данные в формате json в следующем виде: 

In [141]:
{
    'data': [
        {
          'title': 'Разработчик C# (Backend)',
          'work experience': '3-6 лет',
          'salary': '2 500 USD',
          'region': 'Алматы'
        }, {
            
        }
    ]
}

{'data': [{'title': 'Разработчик C# (Backend)',
   'work experience': '3-6 лет',
   'salary': '2 500 USD',
   'region': 'Алматы'},
  {}]}

In [142]:
import requests as req
from bs4 import BeautifulSoup

In [143]:
from time import sleep
from tqdm import tqdm

In [144]:
import json

In [145]:
# not to be defined as DDoS attack
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.114 YaBrowser/22.9.1.1095 Yowser/2.5 Safari/537.36"}

In [146]:
def card_summary(card_parser):
  summary = {'title': None,
          'work experience': None,
          'salary': None,
          'region': None}
  try:
    summary['title'] = (card_parser.find('h1', attrs={'data-qa': 'vacancy-title'}).text)
    summary['work experience'] = (card_parser.find('span', attrs={'data-qa': 'vacancy-experience'}).text)
    summary['salary'] = (parse_salary(card_parser))
    summary['region'] = (card_parser.find('span', attrs={'data-qa': 'vacancy-view-raw-address'}).contents[0])
  except Exception as e:
    pass
  return summary

In [147]:
def parse_salary(card_parser):
  span = card_parser.find('div', attrs={'data-qa': 'vacancy-salary'}).find('span')
  
  if span.attrs['data-qa'] == 'vacancy-salary-compensation-type-undefined':
    return None

  cont = span.contents
  currency = cont[-2]
  salary = 0

  # get elements with salary range
  clear_cont = []
  for elem in cont:
    try:
      int(elem[0])
      clear_cont.append(elem)
    except Exception as e:
      pass
  
  # convert salary to float
  clear_cont = [float(x.replace(u'\xa0', u'')) for x in clear_cont]
  if len(clear_cont) > 1:
    salary = (clear_cont[0]+clear_cont[1])/2
  else:
    salary = clear_cont[0]

  return '{0:8.2f} {1}'.format(salary, currency)

In [148]:
def write_json(new_data, filename='vacancies.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data['data'].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, ensure_ascii=False)

In [149]:
with open('vacancies.json', 'w') as f:
      json.dump({'data':[]}, f, ensure_ascii=False)

In [150]:
for i in tqdm(range(1,10), colour='green'):
  sleep(2.5)
  url = f'https://hh.ru/search/vacancy?page={i}&L_save_area=true&text=python+разработчик&search_field=name&search_field=description&area=113&area=5&area=40&area=9&area=28&area=16&area=1001&area=48&area=97&items_on_page=20'
  response = req.get(url, headers = headers)
  if response.status_code != 200:
    print('Bad response {0}:\t{1}'.format(i, response.status_code))
    continue
  
  vacancy = []
  parser = BeautifulSoup(response.text, 'lxml')
  for tag in parser.find_all('a', attrs={'class': 'serp-item__title'}):
    card_url = tag.get('href')
    sleep(0.8)
    card = req.get(card_url, headers=headers)
    if card.status_code != 200:
      print('Bad card response:\t{0}\t{1}'.format(response.status_code, card_url))
      continue

    card_parser = BeautifulSoup(card.text, 'lxml')
    vacancy.append(card_summary(card_parser))

    write_json(vacancy)

100%|██████████| 9/9 [06:56<00:00, 46.32s/it]


## 2. Палиндром строки
Дана строка X, возвращайте True, если X является палиндромом.

In [166]:
def is_palindrome(input_str):
  if not isinstance(input_str, str): return False
  input_str = input_str.replace(' ', '')
  str_len = len(input_str)
  if str_len == 0: return False
  if str_len == 1: return True

  half_len = str_len//2
  input_str = input_str.lower()
  for i in range(half_len):
    if input_str[i] != input_str[-(i+1)]:
      return False
  
  return True


In [168]:
assert(is_palindrome('taco cat') == True)
assert(is_palindrome('rotator') == True)
assert(is_palindrome('black cat') == False)
assert(is_palindrome('а роза упала на лапу азора') == True)

## 3. Перевод арабского числа в римское
Дано целое положительное число X, необходимо вывести вариант этого числа в римской системе счисления в формате строки.

In [200]:
def roman_numerals(x):
  x = list(str(x))
  x = [int(x_i) for x_i in x]
  while len(x) != 4: 
    x = [0] + x
  roman_x = []
  for i in range(4):
    if x[i] == 0:
      continue
    roman_x.append(create_digit(x[i],i))
  return ''.join(roman_x)

In [201]:
digits = {3: ['I', 'V', 'X'], 2: ['X', 'L', 'C'], 1: ['C', 'D', 'M'], 0: ['M']}

In [202]:
def define_type(x):
  if x < 4:
    return 0
  if x == 4:
    return 1
  if x < 9:
    return 2
  if x == 9:
    return 3

In [203]:
def create_digit(x, pos):
  type_ = define_type(x)
  if type_ == 1:
    return digits[pos][0] + digits[pos][1]
  if type_ == 3:
    return digits[pos][0] + digits[pos][2]

  if type_ == 0:
    return digits[pos][0]*(x)
  if type_ == 2:
    return digits[pos][1]+digits[pos][0]*(x-5)

In [205]:
assert(roman_numerals(3) == 'III')
assert(roman_numerals(9) == 'IX')
assert(roman_numerals(1945) == 'MCMXLV')

## 4. Валидность скобок
Дана строка X, состоящая только из символов '{', '}', '[', ']', '(', ')'. Программа должна вывести True, в том случае если все открытые скобки закрыты.   

Например: '[()]{}', все открытые скобки закрыты закрывающимися скобками, потому вывод будет True.   
В случае же, если строка будет похожа на: '{{{}', то вывод будет False, т.к. не все открытые скобки закрыты.

In [207]:
def is_closed(bracket_string):
  brackets = {')':'(', ']':'[', '}':'{'}
  open_brackets = []
  for sym in bracket_string:
    if sym in brackets.values():
      open_brackets.append(sym)
    elif len(open_brackets) > 0 and open_brackets[-1] == brackets[sym]:
      open_brackets.pop()
    else:
      return False
  return len(open_brackets) == 0

In [210]:
assert(is_closed('[()]{}') == True)
assert(is_closed('{{{}') == False)
assert(is_closed('{[}]') == False)

## 5. Умножить два бинарных числа в формате строк
 На вход подаются две строки X1 и X2, содержащие бинарные числа.  
 Необходимо вывести их бинарное произведение в формате строки.

In [219]:
def mult(a,b):
  a = int(a, 2)
  b = int(b, 2)
  return str(bin(a*b))[2:]

In [221]:
assert(mult('111', '101') == '100011')